In [1]:
# import tensorflow as tf
# from tensorflow.keras import backend as k
# from tensorflow import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers.recurrent import SimpleRNN
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.decomposition import PCA
from data_pretreat import handle_data
from PIL import Image

k.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
train_x,train_y,test_x,test_y=handle_data(train_scale=0.6)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

num_classes=16

(6143, 200)
(6143, 1)
(4106, 200)
(4106, 1)


In [3]:
#降维
def pca(x,n=196):
    pca=PCA(n_components=n)
    new_train_x=pca.fit_transform(x)
    return new_train_x

In [4]:
data_all=np.r_[train_x,test_x]
data_all=data_all/np.max(data_all)
data_all=pca(data_all)
train_x=data_all[0:train_x.shape[0],:]
test_x=data_all[train_x.shape[0]:,:]
print(train_x.shape)
print(test_x.shape)
train_x=train_x.reshape(train_x.shape[0],14,14)
#train_x=np.expand_dims(train_x,axis=3)
test_x=test_x.reshape(test_x.shape[0],14,14)
#test_x=np.expand_dims(test_x,axis=3)

(6143, 196)
(4106, 196)


In [5]:
#one-hot
train_y=np_utils.to_categorical(train_y)[:,1:17]
test_y =np_utils.to_categorical(test_y)[:,1:17]
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(6143, 14, 14)
(6143, 16)
(4106, 14, 14)
(4106, 16)


In [6]:
input_size = 14    # 数据长度-一行有14个像素
time_steps = 14    # 序列长度-一共有14行
cell_size = 256     # 隐藏层cell个数

In [7]:
def s_rnn_model(time_steps,input_size,num_classes=16):
    model = Sequential()
    model.add(SimpleRNN(units=256, input_shape=(time_steps, input_size)))
    model.add(Dense(128, activation='tanh'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

In [8]:
s_rnn_model=s_rnn_model(time_steps,input_size,num_classes)
s_rnn_model.compile(loss='categorical_crossentropy',
                  #optimizer=keras.optimizers.Adadelta(),
                  optimizer=Adam(),
                  metrics=['accuracy'])

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


s_rnn_model.fit(train_x, train_y,
            batch_size=32,
            epochs=100,
            verbose=1)


print("Saving model to disk \n")
path="s_rnn_model.h5"
s_rnn_model.save(path)

loss,accuracy=s_rnn_model.evaluate(test_x, test_y)
print('loss:', loss)
print('accuracy:', accuracy)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 256)               69376     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
Total params: 104,336
Trainable params: 104,336
Non-trainable params: 0
_________________________________________________________________
(6143, 14, 14)
(6143, 16)
(4106, 14, 14)
(4106, 16)
Instructions for updating:
Us

6143/6143 [==============================] - 2s 253us/step - loss: 0.3126 - acc: 0.8924
Epoch 65/100
6143/6143 [==============================] - 2s 258us/step - loss: 0.2677 - acc: 0.9074
Epoch 66/100
6143/6143 [==============================] - 2s 261us/step - loss: 0.2396 - acc: 0.9134
Epoch 67/100
6143/6143 [==============================] - 2s 253us/step - loss: 0.2514 - acc: 0.9162
Epoch 68/100
6143/6143 [==============================] - 2s 254us/step - loss: 0.2938 - acc: 0.8979
Epoch 69/100
6143/6143 [==============================] - 2s 254us/step - loss: 0.2708 - acc: 0.9038
Epoch 70/100
6143/6143 [==============================] - 1s 243us/step - loss: 0.2726 - acc: 0.9067
Epoch 71/100
6143/6143 [==============================] - 2s 248us/step - loss: 0.2758 - acc: 0.9038
Epoch 72/100
6143/6143 [==============================] - 2s 252us/step - loss: 0.2717 - acc: 0.9023
Epoch 73/100
6143/6143 [==============================] - 2s 250us/step - loss: 0.2423 - acc: 0.9134
Epo